<a href="https://colab.research.google.com/github/shail2401/ResumeParser/blob/main/Resume_str.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [13]:
# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [50]:
resumeDataSet = pd.read_csv('Resume.csv', encoding='latin1')
resumeDataSet['Cleaned_Resume'] = ' '
resumeDataSet.head()

,ID,Resume_str,Resume_html,Category,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Cleaned_Resume
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [51]:
# Save the modified DataFrame to a new CSV file
resumeDataSet.to_csv('preprocessed_dataset.csv', index=False)

In [36]:
# Define categories and associated skills
category_skills = {
    'INFORMATION-TECHNOLOGY': ['python', 'java', 'sql', 'javascript', 'html', 'css', 'git', 'docker', 'aws'],
    'BUSINESS-DEVELOPMENT': ['sales', 'marketing', 'business strategy', 'customer relations', 'negotiation'],
    'FINANCE': ['accounting', 'financial analysis', 'budgeting', 'auditing', 'risk management', 'investment'],
    'ADVOCATE': ['legal research', 'litigation', 'legal writing', 'contract negotiation', 'legal advice'],
    'ACCOUNTANT': ['accounting', 'bookkeeping', 'financial reporting', 'taxation', 'auditing'],
    'ENGINEERING': ['mechanical engineering', 'electrical engineering', 'civil engineering', 'software engineering', 'project management'],
    'AVIATION': ['pilot', 'aircraft maintenance', 'air traffic control', 'aviation safety', 'flight operations'],
    'FITNESS': ['personal training', 'nutrition', 'exercise physiology', 'fitness coaching', 'strength training'],
    'CHEF': ['culinary arts', 'food safety', 'menu planning', 'culinary techniques', 'restaurant management'],
    'SALES': ['sales strategy', 'customer relationship management', 'negotiation', 'prospecting', 'cold calling'],
    'BANKING': ['banking operations', 'financial services', 'credit analysis', 'loan processing', 'risk management'],
    'HEALTHCARE': ['medical terminology', 'patient care', 'medical coding', 'healthcare administration', 'nursing'],
    'CONSULTANT': ['management consulting', 'business analysis', 'strategy consulting', 'market research', 'project management'],
    'CONSTRUCTION': ['construction management', 'building codes', 'project management', 'construction safety', 'estimating'],
    'PUBLIC-RELATIONS': ['media relations', 'public speaking', 'press releases', 'event planning', 'crisis management'],
    'HR': ['recruitment', 'employee relations', 'performance management', 'training and development', 'human resources management'],
    'DESIGNER': ['graphic design', 'web design', 'user experience design', 'creative thinking', 'branding'],
    'ARTS': ['fine arts', 'performing arts', 'visual arts', 'art history', 'creative expression'],
    'TEACHER': ['lesson planning', 'curriculum development', 'classroom management', 'educational technology', 'student assessment'],
    'APPAREL': ['fashion design', 'textile design', 'garment construction', 'fashion merchandising', 'fashion marketing'],
    'DIGITAL-MEDIA': ['social media marketing', 'content creation', 'digital advertising', 'video production', 'content strategy'],
    'AGRICULTURE': ['crop management', 'livestock management', 'agribusiness', 'sustainable agriculture', 'farm equipment operation'],
    'AUTOMOBILE': ['automotive repair', 'vehicle maintenance', 'automotive engineering', 'automotive technology', 'vehicle diagnostics'],
    'BPO': ['business process outsourcing', 'customer service', 'data entry', 'call center operations', 'quality assurance']
}


In [2]:
# Load the dataset
resume_data = pd.read_csv('preprocessed_dataset.csv')

In [52]:
from google.colab import files
files.download('preprocessed_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
# Initialize a dictionary to store results
results = {'Category': [], 'Skills': [], 'Num_Resumes_with_Skill': []}


In [33]:
def extract_skills(text):
    # Check if the input is a string-like object
    if isinstance(text, str):
        # Tokenize words and perform Part-of-Speech (POS) tagging
        words = word_tokenize(text)
        tagged_words = pos_tag(words)

        # Initialize an empty list to store skills
        skills = []

        # Iterate through tagged words to identify nouns (NN) and proper nouns (NNP)
        for word, tag in tagged_words:
            if tag in ['NN', 'NNP']:  # If word is a noun or proper noun
                skills.append(word)

        return skills
    else:
        # If input is not a string, return an empty list
        return []


In [6]:
# Display the DataFrame with extracted skills
print(resume_data.head())

         ID                                         Resume_str  \
0  16852973  hr administratormarketing associate hr adminis...   
1  22323967  hr specialist us hr operations summary versati...   
2  33176873  hr director summary over 20 years experience i...   
3  27018550  hr specialist summary dedicated driven and dyn...   
4  17812897  hr manager skill highlights hr skills hr depar...   

                                         Resume_html Category  
0  hr administratormarketing associate hr adminis...       HR  
1  hr specialist us hr operations summary versati...       HR  
2  hr director summary over 20 years experience i...       HR  
3  hr specialist summary dedicated driven and dyn...       HR  
4  hr manager skill highlights hr skills hr depar...       HR  


In [24]:
# Function to extract skills based on category
def extract_category_skills(text, category):
    skills = []
    category_skills_list = category_skills.get(category, [])  # Get skills for the specific category
    # Extract skills from the text based on the category-specific skill list
    for skill in category_skills_list:
        if skill.lower() in text.lower():  # Case-insensitive match
            skills.append(skill)
    return skills

In [38]:
# Iterate over categories
for category, skills in category_skills.items():
    # Filter the dataframe for the current category
    category_data = resume_data[resume_data['Category'] == category]

    # Iterate over skills for the current category
    for skill in skills:
        # Count the number of resumes containing the current skill
        num_resumes_with_skill = category_data['Resume_str'].str.contains(skill, case=False, na=False).sum()

        # Append results to the dictionary
        results['Category'].append(category)
        results['Skills'].append(skill)
        results['Num_Resumes_with_Skill'].append(num_resumes_with_skill)


In [40]:
# Save the DataFrame to a CSV file
results_df.to_csv('skills_count_category.csv', index=False)


In [39]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)


In [18]:
# Display the categories and the number of resumes belonging to each category
print('Displaying the categories and the number of resumes belonging to each category:')
print(category_counts_df)

Displaying the categories and the number of resumes belonging to each category:
                  Category  Category_Count
0   INFORMATION-TECHNOLOGY             120
1     BUSINESS-DEVELOPMENT             120
2                  FINANCE             118
3                 ADVOCATE             118
4               ACCOUNTANT             118
5              ENGINEERING             118
6                 AVIATION             117
7                  FITNESS             117
8                     CHEF             117
9                    SALES             116
10                 BANKING             115
11              HEALTHCARE             115
12              CONSULTANT             115
13            CONSTRUCTION             112
14        PUBLIC-RELATIONS             110
15                      HR             110
16                DESIGNER             107
17                    ARTS             103
18                 TEACHER             102
19                 APPAREL              97
20           DIGI

In [10]:
# Example DataFrame with 'Resume_str' column
data = {'Resume_str': ['I am proficient in Python, Java, and SQL. I have experience with machine learning and data analysis.']}

In [17]:
# Convert value counts to a DataFrame
category_counts_df = category_counts.reset_index()
category_counts_df.columns = ['Category', 'Category_Count']


In [11]:
df = pd.DataFrame(data)

In [14]:
# Apply the extract_skills function to each resume
df['Skills'] = df['Resume_str'].apply(extract_skills)

In [21]:
# Count the number of resumes containing each skill
skills_count = df['Skills'].explode().value_counts().reset_index()
skills_count.columns = ['Skill', 'Count']

In [22]:
from google.colab import files
# Save the result to a CSV file
skills_count.to_csv('skills_count.csv', index=False)

# Download the CSV file
files.download('skills_count.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
print(df['Skills'])

0    [Python, Java, SQL, experience, machine, learn...
Name: Skills, dtype: object


In [16]:
# Get the value counts of categories
category_counts = resume_data['Category'].value_counts()

In [20]:
from google.colab import files

# Save the DataFrame to a CSV file
category_counts_df.to_csv('category_counts.csv', index=False)

# Download the CSV file
files.download('category_counts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
def count_resumes_with_skill(category, skill):
    # Filter the dataset by category
    category_data = resume_data[resume_data['Category'] == category]

    # Apply extract_skills function to each resume in the category
    category_data['Skills'] = category_data['Resume_str'].apply(extract_skills)

    # Count the number of resumes containing the skill
    num_resumes_with_skill = category_data['Skills'].apply(lambda x: skill in x).sum()

    return num_resumes_with_skill

In [29]:
# Define categories
categories = resume_data['Category'].unique()


In [30]:
# Initialize an empty dictionary to store results
results = {'Category': [], 'Skill': [], 'Num_Resumes_with_Skill': []}

In [35]:
for category in categories:
    # Get unique skills for the category
    unique_skills = set()
    category_data = resume_data[resume_data['Category'] == category].copy()  # Create a copy of the DataFrame
    category_data['Skills'] = category_data['Resume_str'].apply(extract_skills)

    # Count the number of resumes containing each skill
    for skill in unique_skills:
        num_resumes_with_skill = count_resumes_with_skill(category, skill)
        results['Category'].append(category)
        results['Skill'].append(skill)
        results['Num_Resumes_with_Skill'].append(num_resumes_with_skill)

In [ ]:
# Create DataFrame from results
skills_count_df = pd.DataFrame(results)

In [ ]:
# Save the result to a CSV file
skills_count_df.to_csv('skills_count.csv', index=False)

In [42]:
from google.colab import files


# Download the CSV file
files.download('skills_count_category.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>